In [14]:
import numba
import numpy as np
from functools import reduce
from operator import mul

from minitorch.autodiff import tensor_functions as tf
from minitorch.autodiff.fast_tensor_ops import FastOps
from minitorch.autodiff.tensor_ops import SimpleOps, TensorBackend
from minitorch import testing

In [15]:
testing.MathTestTensor._comp_testing()

([('add_constant',
   <function minitorch.testing.MathTest.add_constant(x: 'T') -> 'T'>,
   <function minitorch.testing.MathTest.add_constant(x: 'T') -> 'T'>),
  ('complex',
   <function minitorch.testing.MathTest.complex(x: 'T') -> 'T'>,
   <function minitorch.testing.MathTestVariable.complex(x: 'T') -> 'T'>),
  ('cube',
   <function minitorch.testing.MathTest.cube(x: 'T')>,
   <function minitorch.testing.MathTest.cube(x: 'T')>),
  ('divide',
   <function minitorch.testing.MathTest.divide(x: 'T') -> 'T'>,
   <function minitorch.testing.MathTest.divide(x: 'T') -> 'T'>),
  ('exp',
   <function minitorch.testing.MathTest.exp(x: 'T') -> 'T'>,
   <function minitorch.testing.MathTestVariable.exp(x: 'T') -> 'T'>),
  ('explog',
   <function minitorch.testing.MathTest.explog(x: 'T') -> 'T'>,
   <function minitorch.testing.MathTestVariable.explog(x: 'T') -> 'T'>),
  ('inv',
   <function minitorch.testing.MathTest.inv(x: 'T')>,
   <function minitorch.testing.MathTest.inv(x: 'T')>),
  ('log',
   

In [4]:
FastBackend = TensorBackend(FastOps)
SimpleBackend = TensorBackend(SimpleOps)

In [5]:
t_simple = tf.rand(shape=(3, 2), backend=SimpleBackend)
t_fast = tf.rand(shape=(3, 2), backend=FastBackend)

In [7]:
t_simple


[
	[0.27896 0.83176]
	[0.17914 0.45068]
	[0.31804 0.36079]
]

In [13]:
t_simple + 6


[
	[6.27896 6.83176]
	[6.17914 6.45068]
	[6.31804 6.36079]
]

In [9]:
t_fast


[
	[0.75765 0.86154]
	[0.47548 0.71621]
	[0.13738 0.77017]
]


[
	[6.75765 6.86154]
	[6.47548 6.71621]
	[6.13738 6.77017]
]